# 가위바위보 분류기 만들기


In [2]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler

print(tf.__version__)
print(np.__version__)

2.6.0
1.22.2


In [1]:
from PIL import Image
import glob
import os

print("PIL 라이브러리 import 완료!")

PIL 라이브러리 import 완료!


In [ ]:
def resize_images(img_path, target_path):
    images=glob.glob(img_path + "/*.jpg")  
    
    print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장.
    target_size=(56, 56)
    for img in images:
        old_img=Image.open(img)
        img_name = old_img.filename.split('/')[-1]
        new_img=old_img.resize(target_size,Image.ANTIALIAS)
        new_img.save(target_path+img_name, "JPEG")
        
    print(len(images), " images resized.")

## Save Resized Training set

In [22]:
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_train/scissor/"
target_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_train_resized/scissor/"
resize_images(image_dir_path, target_path)

print("가위 이미지 resize 완료!")

521  images to be resized.
521  images resized.
가위 이미지 resize 완료!


In [25]:
# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_train/rock/"
target_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_train_resized/rock/"

resize_images(image_dir_path, target_path)
print("바위 이미지 resize 완료!")

408  images to be resized.
408  images resized.
바위 이미지 resize 완료!


In [26]:
# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_train/paper/"
target_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_train_resized/paper/"

resize_images(image_dir_path, target_path)
print("보 이미지 resize 완료!")

506  images to be resized.
506  images resized.
보 이미지 resize 완료!


## Save Resized Test set

In [27]:
# 테스트용 데이터 가위,바위,보에 대하여 이미지 사이즈 조정하기

# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_test/scissor/"
target_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_test_resized/scissor/"
resize_images(image_dir_path, target_path)

print("test용 가위 이미지 resize 완료!")

# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_test/rock/"
target_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_test_resized/rock/"
resize_images(image_dir_path, target_path)

print("test용 바위 이미지 resize 완료!")

# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서 
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_test/paper/"
target_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_test_resized/paper/"
resize_images(image_dir_path, target_path)

print("test용 보 이미지 resize 완료!")

100  images to be resized.
100  images resized.
test용 가위 이미지 resize 완료!
100  images to be resized.
100  images resized.
test용 바위 이미지 resize 완료!
100  images to be resized.
100  images resized.
test용 보 이미지 resize 완료!


# Load dataset

In [3]:
import numpy as np

def load_data(img_path, number_of_data=1435):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=56 # 사이즈를 좀 키워서 해보자
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)
    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        img = img.reshape(img_size, img_size, color)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        img = img.reshape(img_size, img_size, color)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        img = img.reshape(img_size, img_size, color)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels


def load_data2(img_path_, number_of_data_=300):  
    # 가위, 바위, 보 이미지 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    img_size_ = 56
    color_ = 3
    imgs_ = np.zeros(number_of_data_ * img_size_ * img_size_ * color_, dtype=np.int32).reshape(number_of_data_, img_size_, img_size_, color_)
    labels_ = np.zeros(number_of_data_, dtype=np.int32)

    idx = 0
    for file in glob.iglob(img_path_+'/scissor/*.jpg'):
        img_ = np.array(Image.open(file), dtype=np.int32)
        img_ = img_.reshape(img_size_, img_size_, color_)
        imgs_[idx, :, :, :] = img_    # 데이터 영역에 이미지 행렬을 복사
        labels_[idx] = 0   # 가위 : 0
        idx += 1

    for file in glob.iglob(img_path_+'/rock/*.jpg'):
        img_ = np.array(Image.open(file), dtype=np.int32)
        img_ = img_.reshape(img_size_, img_size_, color_)
        imgs_[idx, :, :, :] = img_    # 데이터 영역에 이미지 행렬을 복사
        labels_[idx] = 1   # 바위 : 1
        idx += 1  
    
    for file in glob.iglob(img_path_+'/paper/*.jpg'):
        img_ = np.array(Image.open(file), dtype=np.int32)
        img_ = img_.reshape(img_size_, img_size_, color_)
        imgs_[idx, :, :, :] = img_    # 데이터 영역에 이미지 행렬을 복사
        labels_[idx] = 2   # 보 : 2
        idx += 1
        
    print("학습데이터(x_test)의 이미지 개수는", idx, "입니다.")
    return imgs_, labels_

In [55]:
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_train_resized"
(x_train_1, y_train_1)=load_data(image_dir_path)

print("x_train shape: {}".format(x_train_1.shape))
print("y_train shape: {}".format(y_train_1.shape))

학습데이터(x_train)의 이미지 개수는 1435 입니다.
x_train shape: (1435, 56, 56, 3)
y_train shape: (1435,)


In [56]:
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_test_resized"
(x_train_2, y_train_2)=load_data2(image_dir_path)

print("x_test shape: {}".format(x_train_2.shape))
print("y_test shape: {}".format(y_train_2.shape))

학습데이터(x_test)의 이미지 개수는 300 입니다.
x_test shape: (300, 56, 56, 3)
y_test shape: (300,)


In [62]:
x_train = np.concatenate((x_train_1, x_train_2))
y_train = np.concatenate((y_train_1, y_train_2))

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

x_train shape: (1735, 56, 56, 3)
y_train shape: (1735,)


In [63]:
from sklearn.model_selection import train_test_split

In [64]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=300, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [67]:
x_train_norm = (x_train / 255.0) - 0.5
x_test_norm = (x_test / 255.0) - 0.5

## Training

In [73]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# model 만들기
# Hint! model의 입력/출력부에 특히 유의
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(56,56,3), kernel_regularizer='l2'))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer='l2'))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu', kernel_regularizer='l2'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 54, 54, 32)        896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 27, 27, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 25, 25, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                589888    
_________________________________________________________________
dense_11 (Dense)             (None, 3)                

In [74]:
optimizer = keras.optimizers.Adam(
    learning_rate=0.001,
)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 훈련
model.fit(x_train_norm, y_train, epochs=20, batch_size=100, validation_split=0.2) 

Epoch 1/20
12/12 [==============================] - 7s 542ms/step - loss: 2.2035 - accuracy: 0.5523 - val_loss: 1.6138 - val_accuracy: 0.5749
Epoch 2/20
12/12 [==============================] - 6s 536ms/step - loss: 1.2836 - accuracy: 0.7195 - val_loss: 1.0475 - val_accuracy: 0.8571
Epoch 3/20
12/12 [==============================] - 6s 531ms/step - loss: 0.9366 - accuracy: 0.8389 - val_loss: 0.8499 - val_accuracy: 0.8780
Epoch 4/20
12/12 [==============================] - 6s 529ms/step - loss: 0.7417 - accuracy: 0.9085 - val_loss: 0.7054 - val_accuracy: 0.9268
Epoch 5/20
12/12 [==============================] - 6s 538ms/step - loss: 0.6719 - accuracy: 0.9155 - val_loss: 0.7252 - val_accuracy: 0.8850
Epoch 6/20
12/12 [==============================] - 6s 530ms/step - loss: 0.6141 - accuracy: 0.9286 - val_loss: 0.6375 - val_accuracy: 0.9233
Epoch 7/20
12/12 [==============================] - 6s 532ms/step - loss: 0.5362 - accuracy: 0.9547 - val_loss: 0.5370 - val_accuracy: 0.9233
Epoch 

## Test

In [75]:
# model 추론 결과
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss : {}".format(test_loss))
print("test_accuracy : {}".format(test_accuracy))

10/10 - 0s - loss: 0.2689 - accuracy: 0.9900
test_loss : 0.2688635289669037
test_accuracy : 0.9900000095367432


In [76]:
preds = np.argmax(model.predict(x_test_norm), axis=1)

In [77]:
preds

array([1, 0, 1, 2, 1, 2, 0, 2, 0, 1, 1, 1, 0, 0, 2, 2, 1, 2, 2, 0, 1, 2,
       1, 2, 1, 2, 1, 1, 1, 1, 2, 0, 2, 1, 2, 0, 1, 2, 0, 1, 0, 2, 1, 1,
       0, 0, 2, 1, 2, 2, 1, 2, 2, 0, 2, 1, 1, 1, 0, 2, 0, 0, 1, 2, 2, 1,
       2, 2, 0, 2, 2, 2, 2, 0, 2, 1, 0, 0, 1, 0, 0, 1, 0, 2, 1, 1, 0, 1,
       0, 0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 2, 0, 0,
       0, 0, 1, 0, 2, 1, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 1, 0, 1, 1, 2,
       0, 1, 2, 0, 2, 1, 0, 2, 0, 2, 0, 0, 2, 1, 2, 2, 1, 2, 0, 2, 0, 0,
       2, 0, 2, 1, 2, 1, 2, 0, 0, 0, 1, 1, 2, 2, 0, 1, 0, 2, 1, 0, 2, 0,
       1, 1, 0, 2, 0, 0, 0, 0, 2, 0, 2, 2, 1, 2, 2, 1, 0, 2, 0, 2, 1, 0,
       0, 1, 2, 1, 1, 2, 2, 1, 0, 0, 2, 0, 2, 2, 0, 2, 2, 2, 0, 1, 1, 2,
       0, 2, 0, 1, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 1, 0, 0, 1, 0, 1, 0, 2,
       2, 0, 1, 1, 0, 1, 2, 0, 0, 2, 0, 1, 2, 0, 1, 1, 2, 1, 0, 0, 0, 1,
       2, 1, 0, 1, 0, 0, 2, 0, 1, 0, 2, 1, 1, 2, 2, 2, 0, 0, 2, 0, 0, 2,
       1, 1, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 1, 2])

<br>  
<br>  
<br>  
<br>  

---

## 회고
l2 정규화도 해보고, 224x224사이즈 이미지를 28x28로 줄이니 사진이 너무 깨지기 때문에 56x56으로 해봤더니 사진이 좀 덜 깨졌다. 이미지 데이터를 다시 모을 수는 없으니 사진 사이즈를 키워보고 했더니 모델이 잘 인식한 것 같다...!   <br>  
여기저기 물어보고 찾아보면서 도움을 받아 해봤더니 재밌는 프로젝트였다  <br>  
그래도 혼자 코드를 짜보라고 하면 아직은 어려운 것 같다 :(